<a href="https://colab.research.google.com/github/Gabrielalopes15/Treinos-em-python/blob/Minha-vers%C3%A3o-da-imers%C3%A3o/Minha_vers%C3%A3o_de_agentes_de_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -q --upgrade langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00


Importação da API

In [3]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI

GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')

Conexão com Gemini
temperature "sentimento"

In [4]:
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 1.0,
    api_key = GOOGLE_API_KEY
)

In [16]:
TRIAGEM_BETHA = (
    "Você é um triador de service desk de um sistema de orgão publico e da os primeiros atendimentos basicos antes de passar pro atendente técnico. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "modulo": "COMPRAS" | "CONTRATOS" | "OBRAS" | "PATRIMONIO" | "ALMOXARIFADO" | "FROTAS",\n'
    '  "submodulo": "PNCP" | "SICOM" | "INTEGRACAO_LICITACAO" | "INTEGRACAO_CONTABIL" | "INTEGRACAO_ALMOXARIFADO" | "INTEGRACAO_PATRIMONIO" | "GERAL",\n'
    '  "prioridade": "CRITICA" | "ALTA" | "MEDIA" | "BAIXA",\n'
    '  "descricao_resumida": "Resumo em até 10 palavras"\n'
    '  "campos adicionais": ["..."]\n'
    "}\n"
    "\n"
    "Regras :\n"
    "- Sempre escolha um MODULO principal de acordo com o contexto da mensagem.\n"
    "  * COMPRAS → licitações, pregão, editais, propostas, PNCP, SICOM. Se o assunto for publicação, PNCP e SICOM é urgente.\n"
    "  * CONTRATOS → termos, aditivos, execução, integração contábil, solicitação, integração com almoxarifado, integração com patrimônio, AM.\n"
    "  * OBRAS → medições, fiscalização, planejamento de obras, SISOP, SICOM.\n"
    "  * PATRIMONIO → bens móveis, tombamento, depreciação, integração com almoxarifado.\n"
    "  * ALMOXARIFADO → estoque, requisições, integrações com patrimônio.\n"
    "  * FROTAS → veículos, abastecimento, motoristas, manutenção, despesas, ordem de abastecimento.\n"
    "\n"
    "- SUBMODULOS:\n"
    "  * PNCP → quando citar 'PNCP' ou 'plataforma nacional de contratações públicas'.\n"
    "  * SICOM → quando mencionar integração com 'TCE', 'SICOM', ou relatórios exigidos.\n"
    "  * INTEGRACAO_LICITACAO → quando falar de integração com portais/licitações externas do tipo BNC, LICITANET, LICITAR DIGITAL, BLL. Essa integração tem prioridade media.\n"
    "  * INTEGRACAO_CONTABIL → quando envolver contabilidade, empenhos, financeiro.\n"
    "  * INTEGRACAO_ALMOXARIFADO → quando falar de estoque integrando com patrimônio.\n"
    "  * INTEGRACAO_PATRIMONIO → quando patrimônio e compras/almoxarifado estiverem relacionados.\n"
    "  * GERAL → se não houver integração ou submódulo específico.\n"
    "- PRIORIDADE:\n"
    "  * CRITICA → sistema totalmente indisponível ou vários usuários afetados.\n"
    "  * ALTA → erros que impedem conclusão de tarefas principais.\n"
    "  * MEDIA → problemas que têm alternativa temporária ou afetam poucos usuários.\n"
    "  * BAIXA → dúvidas de uso, solicitações simples ou informativas.\n"
    "\n"
    "- descricao_resumida: Sempre resuma o problema em até 10 palavras, sem perder o sentido.\n"
    "\n"
)


In [17]:
TELAS_BETHA = {
    "COMPRAS": {
        "PNCP": "Executando > Relatórios > Integração com PNCP ou Apertar a tecla F4 > Integração com PNCP",
        "INTEGRACAO_LICITACAO": "Executando > Relatórios ou Apertar a tecla F4",
    },
    "CONTRATOS": {
        "INTEGRACAO_CONTABIL": "Executando > Solicitação de fornecimento > Interação de dados > + Dados",
        "GERAL": "Menu Principal > Contratos"
    }
}


In [18]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  modulo: Literal ["COMPRAS" , "CONTRATOS" , "OBRAS" , "PATRIMONIO" , "ALMOXARIFADO" , "FROTAS"]
  submodulo: Literal ["PNCP" , "SICOM" , "INTEGRACAO_LICITACAO" , "INTEGRACAO_CONTABIL" , "INTEGRACAO_ALMOXARIFADO" , "INTEGRACAO_PATRIMONIO" , "GERAL"]
  prioridade: Literal ["CRITICA" , "ALTA" , "MEDIA" , "BAIXA"]
  campos_adicionais: List[str] = Field(default_factory = list)


In [19]:
llm_triagem = ChatGoogleGenerativeAI(
    model = "gemini-2.5-flash",
    temperature = 1.0,
    api_key = GOOGLE_API_KEY
)

In [20]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_BETHA),
        HumanMessage(content=mensagem)
    ])

    return saida.model_dump()


In [21]:
testes = ["Como incluir contrato?",
          "preciso enviar um pncp, como faço urgente?",
          "preciso cadastrar um fornecedor como faço?",
          "tem um erro no sicom, sobre o aberlic como faço para corrigir?"]

In [22]:
for msg_teste in testes:
  print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Como incluir contrato?
 -> Resposta: {'modulo': 'CONTRATOS', 'submodulo': 'GERAL', 'prioridade': 'BAIXA', 'campos_adicionais': ['Incluir Contrato']}

Pergunta: preciso enviar um pncp, como faço urgente?
 -> Resposta: {'modulo': 'COMPRAS', 'submodulo': 'PNCP', 'prioridade': 'ALTA', 'campos_adicionais': ['Dúvida sobre envio PNCP']}

Pergunta: preciso cadastrar um fornecedor como faço?
 -> Resposta: {'modulo': 'COMPRAS', 'submodulo': 'GERAL', 'prioridade': 'BAIXA', 'campos_adicionais': ['Cadastrar fornecedor no sistema']}

Pergunta: tem um erro no sicom, sobre o aberlic como faço para corrigir?
 -> Resposta: {'modulo': 'COMPRAS', 'submodulo': 'SICOM', 'prioridade': 'ALTA', 'campos_adicionais': ['aberlic']}



palavra chave langchain